In [ ]:
import pandas as pd
df_train=pd.read_csv('train.csv')
train_columns=df_train.columns
train_columns=train_columns[0:6]
df_train_copy=df_train
df_test= pd.read_csv('test.csv', 
                  names =train_columns)
Frame=pd.DataFrame(df_test, columns = train_columns)
# df_test=pd.read_csv('test.csv',header=None)

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
df_test

# 1. Steps of Data Exploration and Preparation

The quality of our inputs decide the quality of your output.So, once we have got our business hypothesis ready, it makes sense to spend lot of time and efforts here.Data exploration, cleaning and preparation can take up to 70% of our total project time.

The steps involved to understand, clean and prepare our data for building our predictive model are:

    
   * Variable Identification
   * Univariate Analysis
   * Bi-variate Analysis
   * Missing values treatment
   * Outlier treatment
   * Variable transformation
   * Variable creation

Finally, we will need to iterate over steps 4 – 7 multiple times before we come up with our refined model.


Let’s now study each stage in  detail:-

## Variable Identification

**First**:Identify Predictor (Input) and Target (output) variables.

In [ ]:
target=df_train['popularity']
df_train=df_train.drop('popularity',axis=1)

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:

df_train['training_set']=1
df_test['training_set']=0
df_full=pd.concat([df_train,df_test])

**Second**:Identify the data type and category of the variables.

In [ ]:
df_full.info()

In [ ]:
df_full

In [ ]:
(df_full.T.apply(lambda x: x.nunique(), axis=1)).sort_values(ascending=False)

## Univariate Analysis

In [ ]:
target.describe()

In [ ]:
import numpy as np
from scipy.stats import iqr
q75, q25 = np.percentile(target, [75 ,25])
print(q75)
print(q25)
iqrs = iqr(target)
print("\nRange: "+str(max(target)-min(target)))
print("\nIQR: "+str(iqrs))
print("\nVariance: "+str(np.var(target)))
print("\nStandard Deviation: "+str(np.sqrt(np.var(target))))

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
sns.distplot(target)

In [ ]:
sns.boxplot(target)

# Bi-variate Analysis

In [ ]:
# Table of survival vs. sex
Zoning = pd.crosstab(index=df_train_copy["popularity"], 
                           columns=df_train_copy["maintainence_cost"],margins=True)

Zoning

In [ ]:
ZoningPercent=Zoning.div(Zoning.ix["All"],axis=1)
ZoningPercent=ZoningPercent.T
ZoningPercent

**2)Stacked Column Chart:** This method is more of a visual form of Two-way table.Since the number of inputs are quite large ,we worked on a different data for this Illustration.

## Chi-Square Test: 

In [ ]:
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(Zoning)
print("Chi Value: "+str(chi2))
print("P value: "+str(p))
print(dof)
expected = pd.DataFrame(expected)
expected

In [ ]:
import scipy.stats as stats
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 10)   

print("Critical value")
print(crit)

# 2. Missing Value Treatment

In [ ]:
(df_full.shape[0]-df_full.apply(lambda x: x.count(), axis=0)).sort_values(ascending=False)

In [ ]:
((df_full.shape[0]-df_full.apply(lambda x: x.count(), axis=0))!=0).sum()

There are in total of 0 columns which have atleast one value missing

In [ ]:
quantitative = [f for f in df_full.columns if df_full.dtypes[f] != 'object']
print(len(quantitative))

quantitative is the  columns which are either float64 or int64.Now we will count the number of zeros in all the columns.

In [ ]:
(df_full.shape[0]-df_full.astype(bool).sum(axis=0)).sort_values(ascending=False)

In [ ]:
import numpy as np   
mean = df_full[quantitative].apply(lambda x:np.mean(x, axis=0))
sd =df_full[quantitative].apply(lambda x:np.std(x, axis=0))
MeanSD = pd.concat([mean, sd], axis=1, keys=['Mean', 'Standard Deviation'])
MeanSD

In [ ]:
print(df_full.shape)

In [ ]:
qualitative = [f for f in df_full.columns if f not in quantitative]

In [ ]:
df_full

In [ ]:
(df_full.shape[0]-df_full.apply(lambda x: x.count(), axis=0)).sort_values(ascending=False)

In [ ]:
df_train=df_full[df_full['training_set']==1]
df_train=df_train.drop('training_set',axis=1)
#df_train = pd.get_dummies(df_train)
print(df_train.shape)


df_test=df_full[df_full['training_set']!=1]
df_test=df_test.drop('training_set',axis=1)
#df_test = pd.get_dummies(df_test)
#print(df_test.shape)

In [ ]:
print(df_train.shape)
print(target.shape)
print(df_test.shape)

In [ ]:
for i in df_full.columns:
    print(i)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=320,n_jobs=-1)
rf.fit(df_train,target)

In [ ]:
import numpy as np
from sklearn import metrics
from sklearn.model_selection import cross_val_score,cross_val_predict
cv_scores =cross_val_score(rf,df_train,target,cv=5)
# Print the 5-fold cross-validation scores
print(cv_scores)
print("Average 5-Fold CV Score: {}".format(np.mean(cv_scores)))
predictions=cross_val_predict(rf,df_train,target,cv=5)
print(predictions.shape)
accuracy=metrics.r2_score(target,predictions)
print(accuracy)
preds=rf.predict(df_test)

## Ensemeble

In [ ]:
import numpy
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
Y=target.values
X=df_train.values
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y,
test_size=validation_size, random_state=seed)
# Test options and evaluation metric
num_folds = 10
seed = 7

In [ ]:
# Spot-Check Algorithms
scoring = 'f1'
models = []
models.append(( ' LR ' , LogisticRegression()))
models.append(( ' LDA ' , LinearDiscriminantAnalysis()))
models.append(( ' KNN ' , KNeighborsClassifier()))
models.append(( ' CART ' , DecisionTreeClassifier()))
models.append(( ' NB ' , GaussianNB()))
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# ensembles
ensembles = []
ensembles.append(( ' AB ' , AdaBoostClassifier()))
ensembles.append(( ' GBM ' , GradientBoostingClassifier()))
ensembles.append(( ' RF ' , RandomForestClassifier()))
ensembles.append(( ' ET ' , ExtraTreesClassifier()))
results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
model = GaussianNB()
model.fit(X_train,Y_train)
prediction = model.predict(X_validation)
print(accuracy_score(Y_validation, prediction))
print(confusion_matrix(Y_validation, prediction))
print(classification_report(Y_validation, prediction))

In [ ]:
model2 = RandomForestClassifier()
model2.fit(X_train,Y_train)
prediction = model2.predict(X_validation)
print(accuracy_score(Y_validation, prediction))

In [ ]:
model3 = ExtraTreesClassifier()
model3.fit(X_train,Y_train)
prediction = model2.predict(X_validation)
print(accuracy_score(Y_validation, prediction))

In [ ]:
model4 = DecisionTreeClassifier()
model4.fit(X_train,Y_train)
prediction = model4.predict(X_validation)
print(accuracy_score(Y_validation, prediction))

In [ ]:
model5=GradientBoostingClassifier()
model5.fit(X_train,Y_train)
prediction = model5.predict(X_validation)
print(accuracy_score(Y_validation, prediction))

In [ ]:
model6=AdaBoostClassifier()
model6.fit(X_train,Y_train)
prediction = model6.predict(X_validation)
print(accuracy_score(Y_validation, prediction))

In [ ]:
model7=KNeighborsClassifier()
model7.fit(X_train,Y_train)
prediction = model7.predict(X_validation)
print(accuracy_score(Y_validation, prediction))

In [ ]:
model8=LogisticRegression()
model8.fit(X_train,Y_train)
prediction = model8.predict(X_validation)
print(accuracy_score(Y_validation, prediction))

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 100
model9 = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold)
print(results.mean())

In [ ]:
model.fit(X_train,Y_train)
prediction = model.predict(X_validation)
print(accuracy_score(Y_validation, prediction))

In [ ]:
prediction=model.predict(df_test)

In [ ]:
my_submission=pd.DataFrame(prediction)
my_submission.to_csv('submission_Bagging.csv',header=None,index=False)

In [ ]:
# Voting Ensemble for Classification
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
# create the sub models
estimators = []
model11 = BaggingClassifier()
estimators.append(('baggingclassifier', model11))
model12 = DecisionTreeClassifier()
estimators.append(('cart', model12))
model13 = ExtraTreesClassifier()
estimators.append(('ETC', model13))
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X_train, Y_train, cv=kfold)
print(results.mean())

In [ ]:
ensemble.fit(X_train,Y_train)
prediction = ensemble.predict(X_validation)
print(accuracy_score(Y_validation, prediction))

In [ ]:
prediction=model.predict(df_test)

In [ ]:
my_submission=pd.DataFrame(prediction)
my_submission.to_csv('submission_Ensemble1.csv',header=None,index=False)